# Move to LION dataset 

In [ ]:
import pandas as pd
import geopandas as gpd

lion_nodes = gpd.read_file('../data/lion/lion.gdb', layer='node')
lion_node_names = gpd.read_file('../data/lion/lion.gdb', layer='node_stname')
lion_altnames = gpd.read_file('../data/lion/lion.gdb', layer='altnames') # Alternate names for streets (e.g. Ave of Americas)
lion_lion = gpd.read_file('../data/lion/lion.gdb', layer='lion')

# lion_nodes.explore()

lion_nodes.merge(
    lion_node_names
        .groupby('NodeId')['StreetName']
        .apply(' & '.join)
        .reset_index(),
    how='left',
    left_on = 'NODEID', right_on = 'NodeId',
    indicator=True
)[['StreetName','NODEID','geometry', 'GLOBALID']].set_index('NODEID').to_csv('../data/test_runs/nyc/lion_intersections.csv', index=False)

In [ ]:
#lion_node_names[lion_node_names['StreetName'].str.contains('JAY ST')].merge(lion_nodes, left_on='NodeId', right_on='NODEID')
#lion_nodes.merge(lion_node_names, left_on='NODEID', right_on='NodeId').explore()
lion_nodes.merge(
    lion_node_names
        .groupby('NodeId')['StreetName']
        .apply(' & '.join)
        .reset_index(),
    how='left',
    left_on = 'NODEID', right_on = 'NodeId',
    indicator=True
)[['StreetName','NODEID','geometry', 'GLOBALID']].set_index('NODEID').to_csv('../data/test_runs/nyc/lion_intersections.csv', index=False)

In [ ]:
# lion_altnames
lion_node_names
lion_lion.columns.tolist()
# lion_lion.geom_type.value_counts() # all multilineStrin

MultiLineString    241934
Name: count, dtype: int64

In [25]:
import os
os.listdir()

['cache',
 'project_data_changes.ipynb',
 'LION.ipynb',
 'caprecon_projects.ipynb',
 'pull_project_documents.ipynb',
 'file_migration_templating.py',
 'project_files.ipynb',
 'project_data.ipynb',
 'imagery_cut_and_stitch.ipynb',
 'LLM_API_testing.ipynb',
 'imagery_cut_and_stitch2.ipynb']

In [4]:
#lion_lion
#lion_node_names
#lion_node_names['StreetName'].value_counts()
lion_nodes.merge(lion_node_names, left_on='NODEID', right_on='NodeId', how='left', indicator=True)['_merge'].value_counts() # Awesome, only 174 no overlap
lion_nodes.merge(lion_node_names, left_on='OBJECTID', right_on='NodeId', how='left', indicator=True)['_merge'].value_counts() # 39376
lion_nodes_merged = (
    lion_nodes
    .merge(lion_node_names, left_on='OBJECTID', right_on='NodeId', how='left', indicator=True)
    .rename(columns={'_merge': 'object_merge'})
    .merge(lion_node_names, left_on='NODEID', right_on='NodeId', how='left', indicator=True)
)


lion_nodes_merged['int_name'] = lion_nodes_merged['StreetName_x'] + ' & ' + lion_nodes_merged['StreetName_y']

In [11]:
import osmnx as ox
dt_bk_boundaries = ox.geocoder.geocode_to_gdf('Downtown Brooklyn, New York, USA')


In [20]:
lion_nodes_merged.crs

lion_nodes_merged.clip(
    dt_bk_boundaries.to_crs('EPSG:2263')
).explore()
